In [1]:
# Import Library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
df = pd.read_csv("churn.csv")
features = df.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis=1)
target = df['Exited']

In [16]:
# Encoding categorical variables
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
features['Geography'] = encoder.fit_transform(features['Geography'])
features['Gender'] = encoder.fit_transform(features['Gender'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [17]:
# Convert to NumPy arrays
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()
X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()


In [18]:
# Gradient Boosting Implementation
n_estimators = 50  # Number of boosting rounds
learning_rate = 0.1  # Learning rate
manual_trees = []

In [19]:
# Initialize predictions with the mean of the target variable
initial_prediction = np.mean(y_train_np)
train_predictions = np.full(len(y_train_np), initial_prediction)

# Boosting loop
for _ in range(n_estimators):
    # Compute residuals (negative gradients)
    residuals = y_train_np - train_predictions

    # Fit an optimized regression tree to the residuals
    tree = OptimizedDecisionTree(max_depth=3, num_thresholds=10)
    tree.fit_and_predict(X_train_np, residuals)
    manual_trees.append(tree)

    # Update predictions with scaled tree predictions
    train_predictions += learning_rate * tree.predict(X_train_np)

In [20]:
# Importing required libraries
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [21]:
# Decision Tree Implementation
class OptimizedDecisionTree:
    """
    A more efficient implementation of a regression tree for Gradient Boosting.
    """

    def __init__(self, max_depth=3, min_samples_split=2, num_thresholds=10):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.num_thresholds = num_thresholds  # Limit thresholds for efficiency
        self.tree = None

    def fit(self, X, y, depth=0):
        """
        Recursively build the tree by splitting on features to minimize variance.
        """
        n_samples, n_features = X.shape
        if depth == self.max_depth or n_samples < self.min_samples_split:
            return np.mean(y)  # Return leaf value (mean of target values)
        
        best_split = self.find_best_split(X, y, n_features)
        if not best_split:
            return np.mean(y)  # No split possible, return leaf value
        
        feature, threshold, left_idx, right_idx = best_split
        left_tree = self.fit(X[left_idx], y[left_idx], depth + 1)
        right_tree = self.fit(X[right_idx], y[right_idx], depth + 1)
        
        return {"feature": feature, "threshold": threshold, "left": left_tree, "right": right_tree}

    def find_best_split(self, X, y, n_features):
        """
        Find the best split for a node by minimizing residual variance.
        Optimize by reducing the number of thresholds tested.
        """
        best_feature, best_threshold = None, None
        best_variance = float("inf")
        best_left_idx, best_right_idx = None, None

        for feature in range(n_features):
            feature_values = X[:, feature]
            # Use percentiles to reduce threshold candidates
            thresholds = np.percentile(feature_values, np.linspace(0, 100, self.num_thresholds))
            for threshold in thresholds:
                left_idx = feature_values <= threshold
                right_idx = feature_values > threshold
                if np.sum(left_idx) == 0 or np.sum(right_idx) == 0:
                    continue
                
                left_variance = np.var(y[left_idx]) * np.sum(left_idx)
                right_variance = np.var(y[right_idx]) * np.sum(right_idx)
                total_variance = left_variance + right_variance

                if total_variance < best_variance:
                    best_variance = total_variance
                    best_feature = feature
                    best_threshold = threshold
                    best_left_idx = left_idx
                    best_right_idx = right_idx

        if best_feature is None:
            return None  # No valid split found
        
        return best_feature, best_threshold, best_left_idx, best_right_idx

    def predict_single(self, x, tree):
        """
        Predict the target value for a single sample by traversing the tree.
        """
        if not isinstance(tree, dict):  # Leaf node
            return tree

        feature = tree["feature"]
        threshold = tree["threshold"]
        if x[feature] <= threshold:
            return self.predict_single(x, tree["left"])
        else:
            return self.predict_single(x, tree["right"])

    def predict(self, X):
        """
        Predict target values for all samples.
        """
        return np.array([self.predict_single(x, self.tree) for x in X])

    def fit_and_predict(self, X, y):
        """
        Fit the tree and set it as the current model.
        """
        self.tree = self.fit(X, y)
        return self.predict(X)

In [22]:
# Final prediction on test data
test_predictions = np.full(len(y_test_np), initial_prediction)
for tree in manual_trees:
    test_predictions += learning_rate * tree.predict(X_test_np)

# Convert predictions to binary (0/1) using 0.5 as the threshold
test_predictions_binary = (test_predictions >= 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test_np, test_predictions_binary)
print(f"Gradient Boosting Accuracy: {accuracy:.2f}")

Gradient Boosting Accuracy: 0.86
